In [30]:
import pm4py
import pandas as pd

# pd.options.display.max_rows = 4000

In [31]:
log = pd.read_csv("ircc_uOttawa.csv")

/var/folders/b6/7128wh613rqcbppftg_0hb2h0000gn/T/ipykernel_17781/1794484339.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  log = pd.read_csv("ircc_uOttawa.csv")


In [32]:
log = log[['case_id', 'event', 'timestamp', 'act_upd_by', 'cor']]
log = log.rename(columns={ 'case_id' : 'case:concept:name', 'event': 'concept:name', 'timestamp': 'time:timestamp' })
log['case:concept:name'] = log['case:concept:name'].astype(str)
log['time:timestamp'] = pd.to_datetime(log['time:timestamp'])
log

,case:concept:name,concept:name,time:timestamp,act_upd_by,cor
0,1,Misrepresentation - NIL,2024-06-03 00:00:00.000,CLIENT,Vietnam
1,1,Eligibility Assessment - NIL,2024-06-03 00:00:00.000,CLIENT,Vietnam
2,1,Final Assessment - NIL,2024-06-03 00:00:00.000,CLIENT,Vietnam
3,1,Received - Date,2024-06-03 00:00:00.000,CLIENT,Vietnam
4,1,Application Status - Prospective,2024-06-03 09:20:37.327,EAIMEL,Vietnam
...,...,...,...,...,...
543278,7734,Other Reqs Assessment - NIL,2024-06-17 07:46:39.433,SS36358,Belgium
543279,7734,Eligibility Assessment - Review Required,2024-06-26 08:11:25.043,SS36358,Belgium
543280,7734,Eligibility Assessment - Failed,2024-06-28 10:45:09.050,SS36358,Belgium
543281,7734,Final Assessment - Refused,2024-06-28 10:45:09.587,SS36358,Belgium


In [33]:
from mine_utils import log_subset_horizontal

log = log_subset_horizontal(log, 0.02)
log 

,case:concept:name,concept:name,time:timestamp,act_upd_by,cor
0,1,Misrepresentation - NIL,2024-06-03 00:00:00.000,CLIENT,Vietnam
1,1,Eligibility Assessment - NIL,2024-06-03 00:00:00.000,CLIENT,Vietnam
2,1,Final Assessment - NIL,2024-06-03 00:00:00.000,CLIENT,Vietnam
3,1,Received - Date,2024-06-03 00:00:00.000,CLIENT,Vietnam
4,1,Application Status - Prospective,2024-06-03 09:20:37.327,EAIMEL,Vietnam
...,...,...,...,...,...
10904,155,Medical Assessment - Passed,2024-06-20 13:15:00.110,KC14486,China
10905,155,Eligibility Assessment - Recommend Interview,2024-06-20 18:08:57.550,KC14486,China
10906,155,Final Assessment - Approved,2024-06-25 17:26:44.193,KC14486,China
10907,155,Eligibility Assessment - Passed,2024-06-25 17:26:44.197,KC14486,China


In [34]:
# from mine_utils import log_subset_vertical

# log = log_subset_vertical(log, 0.25)
# log

In [35]:
log.to_csv("ircc_uOttawa_subset.csv", index=False)

In [ ]:
import pm4py.objects.log.exporter.xes.exporter as xes_export
xes_export.apply(log, "ircc_uOttawa_subset.xes")

In [20]:
import pm4py
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.objects.conversion.process_tree import converter as process_tree_converter

def run_alpha(log):
    return alpha_miner.apply(log)

def run_inductive(log):
    process_tree = inductive_miner.apply(log)
    return process_tree_converter.apply(process_tree)

In [ ]:
from sklearn.cluster import KMeans
from cluster_utils import eval_cluster_metrics

clusterer = KMeans(n_clusters=3, random_state=0, n_init='auto')
# relies on trace-based features (see feature_extraction below)
clust_logs = pm4py.cluster_log(log, sklearn_clusterer=clusterer)
sublogs = [ pm4py.convert_to_dataframe(clust_log) for clust_log in clust_logs ]

eval_cluster_metrics(log, sublogs, run_alpha, show_progress_bar=True)